In [ ]:
# Import necessary modules
from PIL import Image  # PIL is used to work with images
from google.colab import files  # Google Colab's file handling utility

# Function to convert the message (data) into binary
def genData(data):
    newd = []  # List to store binary values of characters in data
    for i in data:
        newd.append(format(ord(i), '08b'))  # Convert each character to binary
    return newd

# Function to modify pixels based on the binary data
def modPix(pix, data):
    datalist = genData(data)  # Generate binary data from the message
    lendata = len(datalist)  # Length of the binary data
    imdata = iter(pix)  # Pixel data iterator

    # Loop through each bit of the data
    for i in range(lendata):
        # Extract 9 pixel values at a time
        pix = [value for value in imdata.__next__()[:3] +
               imdata.__next__()[:3] +
               imdata.__next__()[:3]]

        # Modify the pixel values according to the binary data
        for j in range(0, 8):
            if (datalist[i][j] == '0' and pix[j] % 2 != 0):
                pix[j] -= 1  # Make pixel even if the bit is '0'
            elif (datalist[i][j] == '1' and pix[j] % 2 == 0):
                if pix[j] != 0:
                    pix[j] -= 1  # Make pixel odd if the bit is '1'
                else:
                    pix[j] += 1

        # If it's the last bit, set a flag to stop
        if (i == lendata - 1):
            if (pix[-1] % 2 == 0):
                if pix[-1] != 0:
                    pix[-1] -= 1
                else:
                    pix[-1] += 1
        else:
            if (pix[-1] % 2 != 0):
                pix[-1] -= 1

        pix = tuple(pix)
        yield pix[0:3]  # Yield first 3 pixels
        yield pix[3:6]  # Yield next 3 pixels
        yield pix[6:9]  # Yield last 3 pixels

# Function to encode data into the image
def encode_enc(newimg, data):
    w = newimg.size[0]  # Width of the image
    (x, y) = (0, 0)  # Initial pixel position

    # Modify pixels with the encoded data
    for pixel in modPix(newimg.getdata(), data):
        newimg.putpixel((x, y), pixel)  # Replace pixel with modified pixel
        if x == w - 1:  # Move to next row if at the end of current row
            x = 0
            y += 1
        else:
            x += 1

# Function to encode a message into an image
def encode():
    uploaded = files.upload()  # Upload the image file from local system
    img_name = list(uploaded.keys())[0]  # Get the image filename
    image = Image.open(img_name)  # Open the image file

    data = input("Enter the message to encode: ")  # Message to encode
    if len(data) == 0:
        raise ValueError('Message is empty!')

    newimg = image.copy()  # Create a copy of the image to modify
    encode_enc(newimg, data)  # Encode the message into the image

    # Save the new image with the encoded message
    new_img_name = input("Enter the new image name (with extension): ")
    newimg.save(new_img_name)

    # Download the modified image
    files.download(new_img_name)

# Function to decode the hidden message from an image
def decode():
    uploaded = files.upload()  # Upload the image file from local system
    img_name = list(uploaded.keys())[0]  # Get the image filename
    image = Image.open(img_name)  # Open the image file

    data = ''
    imgdata = iter(image.getdata())  # Create an iterator for the image data

    while True:
        pixels = [value for value in imgdata.__next__()[:3] +
                  imgdata.__next__()[:3] +
                  imgdata.__next__()[:3]]  # Extract 9 pixels

        # Create a binary string from the pixel values
        binstr = ''
        for i in pixels[:8]:
            if i % 2 == 0:
                binstr += '0'  # Append '0' if pixel value is even
            else:
                binstr += '1'  # Append '1' if pixel value is odd

        data += chr(int(binstr, 2))  # Convert binary string to a character
        if pixels[-1] % 2 != 0:  # Stop if the last pixel flag is set
            return data

# Main function to select encoding or decoding
def main():
    choice = int(input(":: Image Steganography ::\n1. Encode\n2. Decode\n"))
    if choice == 1:
        encode()  # Call the encoding function
    elif choice == 2:
        print("Decoded message: " + decode())  # Call the decoding function
    else:
        raise Exception("Invalid choice! Please enter 1 or 2.")

# Run the main function
if __name__ == "__main__":
    main()


:: Image Steganography ::
1. Encode
2. Decode
1


Saving cat-facts.jpg to cat-facts.jpg
Enter the message to encode: This is a confidential notice for the team. Please be aware that the upcoming project review meeting has been rescheduled. The new date for the meeting is Friday, September 27th, at 10 AM. Ensure that all relevant documents and reports are updated and submitted by Thursday evening to facilitate a smooth discussion. If you have any questions or need further clarification, please reach out to the project manager as soon as possible. Thank you for your cooperation.
Enter the new image name (with extension): newcat.jpg


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>